In [1]:
# My character's carrying capacity
my_capacity = 260

In [2]:
# Tools
import pandas as pd
from difflib import get_close_matches

In [3]:
# Get the entire trading details of all 
# bots and NPC's (this will take a moment)
df = pd.read_html(
    'http://greypal.el-fd.org/cgi-bin/querybot?action=Both&item=.%2B',
    header = 0
)[0]

# Don't care about owners or hosters of the bots
df = df.drop(['Hoster', 'Owner'], 1)

In [4]:
# Clean up the amount available to be bought or sold
max_amount = df[df.Amount != '*'].Amount.astype(int).max()
df['Amount'] = df.Amount.apply(
    lambda s: 
    max_amount if s == '*' else int(s)
)

# Take the units ("gc" for "gold coins") off the prices
df['Price'] = df.Price.str[:-2].astype(float)

In [5]:
# Create a dataframe of "opportunities" -- where an item 
# being sold by one bot is being bought by another
oppo = df[df.Action == 'Selling'].merge(
    df[df.Action == 'Buying'],
    on = 'Item'
)

# Filter down to cases where the buying bot will 
# pay more than the price of the selling bot
oppo = oppo[oppo.Price_y > oppo.Price_x]

In [6]:
# Load item weights scraped from el-wiki
weights = pd.read_pickle('weights.pkl')

# Make a dataframe for fuzzy matching item names 
# from the price list to the weights data
matches = oppo[['Item']].drop_duplicates()
matches['ItemMatched'] = matches.Item.apply(
    lambda s:
    get_close_matches(
        word = s, 
        possibilities = weights.index,
        n = 1, cutoff = 0
    )[0]
)
matches = matches.set_index('Item')

# Get the per-unit weights of the opportunities
oppo = oppo.join(
    matches, on = 'Item'
).join(
    weights, on = 'ItemMatched'
)

In [7]:
# Get the per-unit value of each possible exchange between bots
oppo['UnitValue'] = [
    p2-p1 for p1,p2 
    in zip(oppo.Price_x, oppo.Price_y)
]

# The total value of each possible exchange (if all units move)
oppo['TotalValue'] = [
    min(a1,a2)*v for a1,a2,v 
    in zip(
        oppo.Amount_x, oppo.Amount_y, 
        oppo.UnitValue
    )
]

# The number of units that can be transferred in one trip
oppo['LoadSize'] = [
    min(my_capacity // w, a1, a2)
    for w,a1,a2 in zip(
        oppo.Weight, oppo.Amount_x, oppo.Amount_y
    )
]

# The amount that can be gained in one trip
oppo['LoadValue'] = [
    s * v for s,v in zip(oppo.LoadSize, oppo.UnitValue)
]

In [8]:
# View the opportunites with the largest value per load
oppo.sort_values('LoadValue', ascending = False).drop([
    'Action_x', 'Amount_x', 'Price_x', 
    'Action_y', 'Amount_y', 'Price_y', 
    'Weight', 'UnitValue'
], 1).head(20)

,Bot_x,Location_x,Item,Bot_y,Location_y,ItemMatched,TotalValue,LoadSize,LoadValue
9359,BlackAdder,"KJ(C2) 331,199",Tiger fur,Adarah,"WS(C1) 466,425",Tiger Fur,1190.0,34,1190.0
9358,BlackAdder,"KJ(C2) 331,199",Tiger fur,Raised_by_Bats,"TG(C1) 203,148",Tiger Fur,1190.0,34,1190.0
11586,NPC-Ringa,"IotF(C2) 84,94",Ring of Anitora,Adarah,"WS(C1) 466,425",Ring of Anitora,1070.0,214,1070.0
10432,NPC-Clark,"PL(C1) 198,258",Ring of Isla Prima,LumberJack,"Hurquin(C2) 124,80",Ring of Isla Prima,1060.0,106,1060.0
10518,NPC-Karn,"WS(C1) 575,136",Wooden Shield,Adarah,"WS(C1) 466,425",Wooden Shield,600.0,30,600.0
10907,NPC-Tanta,"PA(C2) 84,207",Skull key,TonyStark,"MM(C1) 68,117",Skull key,500.0,50,500.0
10884,NPC-Zantos,"Ida(C2) 370,291",Skeleton key,TonyStark,"MM(C1) 68,117",Skeleton key,500.0,50,500.0
10914,NPC-Zantos,"Ida(C2) 370,291",Skull key,TonyStark,"MM(C1) 68,117",Skull key,500.0,50,500.0
10876,NPC-Tanta,"PA(C2) 84,207",Skeleton key,TonyStark,"MM(C1) 68,117",Skeleton key,500.0,50,500.0
10880,NPC-Zantos,"Ida(C2) 370,291",Skeleton key,Beaver,"PL(C1) 153,96",Skeleton key,750.0,86,430.0


In [9]:
# View the opportunities in prose form 
for _, row in oppo.sort_values(
    'LoadValue', ascending = False
).head(20).iterrows():
    print(
        '%.0fgc: Buy %d %s from %s (%s) for %.0fgc and sell to %s (%s)' % (
            row.LoadValue, row.LoadSize, 
            row.Item, row.Bot_x, row.Location_x, 
            row.Price_x * row.LoadSize,
            row.Bot_y, row.Location_y
        )
    )

1190gc: Buy 34 Tiger fur from BlackAdder (KJ(C2) 331,199) for 2210gc and sell to Adarah (WS(C1) 466,425)
1190gc: Buy 34 Tiger fur from BlackAdder (KJ(C2) 331,199) for 2210gc and sell to Raised_by_Bats (TG(C1) 203,148)
1070gc: Buy 214 Ring of Anitora from NPC-Ringa (IotF(C2) 84,94) for 74900gc and sell to Adarah (WS(C1) 466,425)
1060gc: Buy 106 Ring of Isla Prima from NPC-Clark (PL(C1) 198,258) for 10600gc and sell to LumberJack (Hurquin(C2) 124,80)
600gc: Buy 30 Wooden Shield from NPC-Karn (WS(C1) 575,136) for 3000gc and sell to Adarah (WS(C1) 466,425)
500gc: Buy 50 Skull key from NPC-Tanta (PA(C2) 84,207) for 7500gc and sell to TonyStark (MM(C1) 68,117)
500gc: Buy 50 Skeleton key from NPC-Zantos (Ida(C2) 370,291) for 7500gc and sell to TonyStark (MM(C1) 68,117)
500gc: Buy 50 Skull key from NPC-Zantos (Ida(C2) 370,291) for 7500gc and sell to TonyStark (MM(C1) 68,117)
500gc: Buy 50 Skeleton key from NPC-Tanta (PA(C2) 84,207) for 7500gc and sell to TonyStark (MM(C1) 68,117)
430gc: Buy 86